In [4]:
%pip install langchain
# %pip install langchain-openai
# %pip install langchainhub
# %pip install pypdf
%pip install chromadb
%pip install pandas

# DISCLAIMER: 

```
FOR THIS NOTEBOOK YOU WILL NEED TO BE RUNNING LOCALLY OLLAMA + HAVE THE LLAMA 3.1 MODEL INSTALLED
```

See [here](https://python.langchain.com/v0.2/docs/integrations/chat/ollama/) for how to set that up.

In [5]:
# import os

# THIS NOTEBOOK EXAMPLE WILL USE OLLAMA INSTEAD OF OPENAI!
# Set OPENAI API Key
# os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

A vector database is a way to store these embeddings, these numerical representations that we just discussed.

The pipeline is:
- In coming document
- Create chunks of text from that document
- Embed each chunk
- Store these embeddings

![](2023-07-30-19-32-13.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

In [51]:
import pandas as pd

file_path="./assets-resources/superheroes.csv"
df = pd.read_csv(file_path)

df.head()

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!


In [52]:
df

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!
5,Crimson Cyclone,Super Speed,91,Blazing fast and unstoppable!
6,Aqua Fury,Hydrokinesis,80,Ride the waves of power!
7,Lunar Guardian,Lunar Manipulation,77,Embrace the moon's might!
8,Steel Titan,Super Strength and Durability,95,Indestructible force of nature!
9,Nightblade,Night Vision and Stealth,84,Strike from the shadows!


In [53]:
from langchain.document_loaders.csv_loader import CSVLoader

In [54]:
loader = CSVLoader(file_path)
data = loader.load()
data[:5]

[Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 0}, page_content='Superhero Name: Captain Thunder\nSuperpower: Bolt Manipulation\nPower Level: 90\nCatchphrase: Feel the power of the storm!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 1}, page_content='Superhero Name: Silver Falcon\nSuperpower: Flight and Agility\nPower Level: 85\nCatchphrase: Soar high, fearlessly!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 2}, page_content='Superhero Name: Mystic Shadow\nSuperpower: Invisibility and Illusions\nPower Level: 78\nCatchphrase: Disappear into the darkness!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 3}, page_content='Superhero Name: Blaze Runner\nSuperpower: Pyrokinesis\nPower Level: 88\nCatchphrase: Burn bright and fierce!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 4}, page_content='Superhero Name: Electra-Wave\nSuperpower: Ele

In [55]:
from langchain.chat_models import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    verbose=True,
)

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [57]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

vectordb = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings(model='llama3.1'))

In [58]:
question = "What is the name of the thunder super hero?"
docs = vectordb.similarity_search(question)
len(docs)

4

In [59]:
docs

[Document(metadata={'row': 19, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Silver Seraph\nSuperpower: Telekinesis\nPower Level: 85\nCatchphrase: Move objects with your mind!'),
 Document(metadata={'row': 19, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Silver Seraph\nSuperpower: Telekinesis\nPower Level: 85\nCatchphrase: Move objects with your mind!'),
 Document(metadata={'row': 32, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Frostglide\nSuperpower: Cryo-Teleportation\nPower Level: 85\nCatchphrase: Freeze and glide through space!'),
 Document(metadata={'row': 32, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Frostglide\nSuperpower: Cryo-Teleportation\nPower Level: 85\nCatchphrase: Freeze and glide through space!')]

In [60]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', '{input}')
])


rag_chain_from_docs = (
    {
        'input': lambda x: x['input'],
        'context': lambda x: format_docs(x['context']), 
    }
    | prompt
    | llm
    | StrOutputParser()
)

retriever = vectordb.as_retriever()

# passing the input query to the retriever
retrieve_docs = (lambda x: x['input']) | retriever

qa_chain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)

In [61]:
question = "What is the catch phrase for the super hero with the power of producing balls of fire?"
result = qa_chain.invoke({'input': question})
result

{'input': 'What is the catch phrase for the super hero with the power of producing balls of fire?',
 'context': [Document(metadata={'row': 24, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Blazing Comet\nSuperpower: Fireball Projection\nPower Level: 82\nCatchphrase: Burn brighter than a comet!'),
  Document(metadata={'row': 24, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Blazing Comet\nSuperpower: Fireball Projection\nPower Level: 82\nCatchphrase: Burn brighter than a comet!'),
  Document(metadata={'row': 3, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Blaze Runner\nSuperpower: Pyrokinesis\nPower Level: 88\nCatchphrase: Burn bright and fierce!'),
  Document(metadata={'row': 3, 'source': './assets-resources/superheroes.csv'}, page_content='Superhero Name: Blaze Runner\nSuperpower: Pyrokinesis\nPower Level: 88\nCatchphrase: Burn bright and fierce!')],
 'answer': 'The catchphrase for Blaz

In [62]:
df.iloc[24]

Superhero Name                  Blazing Comet
Superpower                Fireball Projection
Power Level                                82
Catchphrase       Burn brighter than a comet!
Name: 24, dtype: object